In [1]:
import pandas as pd
import time 
import warnings
from tqdm.notebook import trange, tqdm
from os import listdir
from os.path import isfile, join
import glob
import datetime
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
vocab = {
    0 : 'Пн',
    1 : 'Вт',
    2 : 'Ср',
    3 : 'Чт',
    4 : 'Пт',
    5 : 'Сб',
    6 : 'Вс'
}
weather_dict = {
    "akjar":        "Погода/Акъяр.xlsx",
    "borovoe":      "Погода/Боровое.xlsx",
    "bredy":        "Погода/Бреды.xlsx",
    "krasnoepole":  "Погода/Красноеполе.xlsx",
    "orsk":         "Погода/Орск.xlsx",
    "pogoda":       "Погода/Погода p2.xlsx",
    "chelyabinsk":  "Погода/Челябинск.xlsx",
    "youjnostep":   "Погода/Южно-Степное.xlsx",
    "magnitogorsk": "Погода/7Магнитогорскойптицефабрики(Первомайский).xlsx",
}


In [3]:
def create_target_set(target_name, how_to_shift, hours = 48):
    Target_KS = pd.read_csv(target_name+"P_in_out.csv",sep = ";")
    Target_KS["DateTime"] = pd.to_datetime(Target_KS["DateTime"])   
    Target_KS["dayofweek"] = Target_KS["DateTime"].dt.dayofweek
    Target_KS["Pin"] = Target_KS["Pin"].str.replace(",",".").astype('float32')
    Target_KS["Pout"] = Target_KS["Pout"].str.replace(",",".").astype('float32')
    abs = pd.DataFrame()
    abs["DateTime"] = pd.date_range(Target_KS["DateTime"].min(), periods=((Target_KS["DateTime"].max() - Target_KS["DateTime"].min()).seconds/3600 + (Target_KS["DateTime"].max() - Target_KS["DateTime"].min()).days*24 +1), freq="h")
    Target_KS = pd.merge(abs,Target_KS,how='left',on="DateTime")
    Target_KS = Target_KS.ffill() #Тут заполнение пропусков чтобы дискретность была час     
    if how_to_shift == "1":
        Target_KS.loc[Target_KS['dayofweek'] == 0,'DateTime'] -= pd.DateOffset(hours=72)   
        Target_KS.loc[Target_KS['dayofweek'] == 1,'DateTime'] -= pd.DateOffset(hours=96) 
        Target_KS.loc[Target_KS['dayofweek'] == 2,'DateTime'] -= pd.DateOffset(hours=48)
        Target_KS.loc[Target_KS['dayofweek'] == 3,'DateTime'] -= pd.DateOffset(hours=48)
        Target_KS.loc[Target_KS['dayofweek'] == 4,'DateTime'] -= pd.DateOffset(hours=48)
        Target_KS.loc[Target_KS['dayofweek'] == 5,'DateTime'] -= pd.DateOffset(hours=48)
        Target_KS.loc[Target_KS['dayofweek'] == 6,'DateTime'] -= pd.DateOffset(hours=48)
        Target_KS["dayofweek"] = Target_KS["DateTime"].dt.dayofweek
        Target_KS["dayofweek"] = Target_KS["dayofweek"].replace(vocab, regex=True)
        Target_KS.columns = [f'{c}_'+ target_name if c!="DateTime" else c for c in Target_KS]
        Target_KS = Target_KS.drop(Target_KS[(Target_KS["DateTime"].dt.year < 2021)].index)
        return Target_KS
    elif how_to_shift == "2":
        Target_KS['DateTime'] -= pd.DateOffset(hours=hours)
        Target_KS["dayofweek"] = Target_KS["DateTime"].dt.dayofweek
        Target_KS["dayofweek"] = Target_KS["dayofweek"].replace(vocab, regex=True)
        Target_KS.rename(columns={"Pin": "Pin_lag_"+str(hours)+"h", "Pout": "Pout_lag_"+str(hours)+"h"}, inplace = True)
        Target_KS.columns = [f'{c}_'+ target_name if c!="DateTime" else c for c in Target_KS]
        Target_KS = Target_KS.drop(Target_KS[(Target_KS["DateTime"].dt.year < 2021)].index)
        return Target_KS
    else:        
        Target_KS["dayofweek"] = Target_KS["dayofweek"].replace(vocab, regex=True)
        Target_KS.columns = [f'{c}_'+ target_name if c!="DateTime" else c for c in Target_KS]
        return Target_KS
   
    

In [4]:
def prepare_dataset(target_name, how_to_shift_target,hours = 48):
    grs_list = glob.glob("ГРС/*")
    auto_plan_list = glob.glob("Автоплан/*")
    target_set = create_target_set(target_name, how_to_shift_target,hours)
    schema_info = pd.read_csv(target_name+".csv",sep = ";")
    schema_info["DateTime"] = pd.to_datetime(schema_info["DateTime"])
    target_set = pd.merge(target_set,schema_info,how='left',on="DateTime")
    for grs in tqdm(grs_list):
        feature = pd.read_csv(grs, sep = ";")
        feature["DateTime"] = pd.to_datetime(feature["DateTime"])
        for col in feature.columns:
            if col == "DateTime":
                continue 
            try:
                feature[col] = feature[col].astype('float32')
            except ValueError:
                feature[col] = feature[col].str.replace(",",".").astype('float32')        
        feature.columns = [f'{c}_'+ grs.split("\\")[1][:-4] if c!="DateTime" else c for c in feature]
        target_set = pd.merge(target_set,feature,how='left',on="DateTime")
    for auto_list in tqdm(auto_plan_list):
        feature = pd.read_csv(auto_list, sep = ";")
        if "Date" in feature.columns:
            feature["Data"] = feature["Date"]
            feature.drop(columns=["Date"],inplace=True)
        try:
            feature["DateTime"] = pd.to_datetime(feature["Data"])
        except KeyError:
            continue
        feature.drop(columns=["Data"],inplace=True)
        feature["DateTime"] = pd.to_datetime(feature["DateTime"])
        for col in feature.columns:
            if "Login" in feature.columns:
                break
            if col == "DateTime":
                continue 
            try:
                feature[col] = feature[col].astype('float32')
            except ValueError:
                feature[col] = feature[col].str.replace(",",".").astype('float32')   
        feature.columns = [f'{c}_'+ auto_list.split("\\")[1][:-4] if c!="DateTime" else c for c in feature]
        target_set = pd.merge(target_set,feature,how='left',on="DateTime")
    return target_set
        
    

In [5]:
def correct_data_into_value(date):
    if type(date) == type(datetime.datetime(2024, 2, 13, 0, 0)):
        return str(f"{date.day}.{date.month%12}")
    return date

In [6]:
def prepare_weather_dict_dataset():
   weather_dfs = {}
   for key, value in tqdm(weather_dict.items()):
       active_df = pd.ExcelFile(value)
       print(active_df.sheet_names)
       for i in tqdm(active_df.sheet_names):
           if len(active_df.sheet_names) < 2:
               weather_dfs.update({key: {i: pd.read_excel(value,  sheet_name=i)}})
           else:
               for j in active_df.sheet_names:
                   weather_dfs.update({f"{key}_{j}": {j: pd.read_excel(value,  sheet_name=j)}})
   test_df = weather_dfs
   for i in tqdm(weather_dfs.keys()):
       for j in tqdm(weather_dfs[i].keys()):
           print(f"----{i}----")
           first_column = 1
           for k in weather_dfs[i][j]:
               if first_column == 1:
                   first_column = 0
                   continue
               test_df[i][j][k] = weather_dfs[i][j][k].apply(lambda x: correct_data_into_value(x)).astype("float32")
   del weather_dfs
   return test_df

In [7]:
def prepare_and_set_shift_and_lags(target_set, weather_set):
    if target_set["PNA"].dtype == "object":
        target_set["PNA"] = target_set["PNA"].str.replace(",",".").astype('float32')
    else:
        target_set["PNA"] = target_set["PNA"].astype('float32')
    target_set['Day'] = target_set["DateTime"].dt.day
    target_set['Month'] = target_set["DateTime"].dt.month
    target_set['Year'] = target_set["DateTime"].dt.year
    target_set['Hour'] = target_set["DateTime"].dt.hour
    target_set.columns = target_set.columns.str.replace(r"\n", "_", regex=True)
    for i in weather_set.keys():
        for j in weather_set[i].keys():
            buffer = weather_set[i][j].rename(lambda x: x+ ("_"+ i  )*int( x != "Дата"), axis='columns')
            for columns in tqdm(buffer.columns):
                if columns == "Дата":
                    buffer['Day'] = buffer["Дата"].dt.day
                    buffer['Month'] = buffer["Дата"].dt.month
                    buffer['Year'] = buffer["Дата"].dt.year
                    continue
                if buffer[columns].dtype == "float64":
                    buffer[columns] = buffer[columns].astype("float32")
                if buffer[columns].dtype == "float32":
                    buffer[columns + "_rolling_3"] = buffer[columns].rolling(3, min_periods = 1, center = False).sum()
                # buffer["Date"] = pd.to_datetime(buffer["Date"], format=""
            buffer[columns + "_lag_1"] = buffer[columns].shift(1)
            buffer[columns + "_lag_2"] = buffer[columns].shift(2)
            buffer[columns + "_lag_3"] = buffer[columns].shift(3)
            buffer[columns + "_lag_7"] = buffer[columns].shift(7)


            buffer.drop(columns=["Дата"], inplace=True)
            
            target_set = pd.merge(target_set, buffer, on=["Year","Month","Day"])
    del buffer
    target_set.columns = target_set.columns.str.replace(r"\n", "_", regex=True)
    target_set[target_set.select_dtypes(include=['float64']).columns] = target_set[target_set.select_dtypes(include=['float64']).columns].astype("float32")
    return target_set

In [8]:
t_set = prepare_dataset("КС-16", "2")

  0%|          | 0/119 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [9]:
weather_dfs = prepare_weather_dict_dataset()

  0%|          | 0/9 [00:00<?, ?it/s]

['Лист1']


  0%|          | 0/1 [00:00<?, ?it/s]

['Лист1']


  0%|          | 0/1 [00:00<?, ?it/s]

...

----chelyabinsk----


  0%|          | 0/1 [00:00<?, ?it/s]

----youjnostep----


  0%|          | 0/1 [00:00<?, ?it/s]

----magnitogorsk----


In [10]:
result = prepare_and_set_shift_and_lags(t_set, weather_dfs)

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

...

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

In [11]:
result

,DateTime,Object_КС-16,Pin_lag_48h_КС-16,Pout_lag_48h_КС-16,dayofweek_КС-16,Object,Tnv,GPAinwork,PNA,Schema1,...,Минимальная_температура_magnitogorsk_rolling_3,Средняя_температура_magnitogorsk_rolling_3,Атмосферное_давление_magnitogorsk_rolling_3,Скорость_ветра_magnitogorsk_rolling_3,Осадки_magnitogorsk_rolling_3,Эффективная_температура_magnitogorsk_rolling_3,Эффективная_температура_magnitogorsk_lag_1,Эффективная_температура_magnitogorsk_lag_2,Эффективная_температура_magnitogorsk_lag_3,Эффективная_температура_magnitogorsk_lag_7
0,2021-01-01 00:00:00,6010.0,45.700001,45.700001,Пт,6010.0,-10.0,0.0,NaN,0.0,...,-20.4,-15.100000,992.400024,2.0,0.0,-19.4,NaN,NaN,NaN,NaN
1,2021-01-01 01:00:00,6010.0,45.700001,45.700001,Пт,NaN,NaN,NaN,NaN,NaN,...,-20.4,-15.100000,992.400024,2.0,0.0,-19.4,NaN,NaN,NaN,NaN
2,2021-01-01 02:00:00,6010.0,46.000000,46.000000,Пт,6010.0,-11.0,0.0,NaN,0.0,...,-20.4,-15.100000,992.400024,2.0,0.0,-19.4,NaN,NaN,NaN,NaN
3,2021-01-01 03:00:00,6010.0,46.000000,46.000000,Пт,NaN,NaN,NaN,NaN,NaN,...,-20.4,-15.100000,992.400024,2.0,0.0,-19.4,NaN,NaN,NaN,NaN
4,2021-01-01 04:00:00,6010.0,46.099998,46.099998,Пт,6010.0,-11.0,0.0,NaN,0.0,...,-20.4,-15.100000,992.400024,2.0,0.0,-19.4,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32776,2023-09-30 23:00:00,6010.0,39.500000,47.500000,Сб,NaN,NaN,NaN,NaN,NaN,...,17.6,30.099998,2920.399902,7.0,2.0,24.0,8.6,8.4,7.4,13.6
32777,2023-09-30 23:00:00,6010.0,39.500000,47.500000,Сб,NaN,NaN,NaN,NaN,NaN,...,17.6,30.099998,2920.399902,7.0,2.0,24.0,8.6,8.4,7.4,13.6
32778,2023-09-30 23:00:00,6010.0,39.500000,47.500000,Сб,NaN,NaN,NaN,NaN,NaN,...,17.6,30.099998,2920.399902,7.0,2.0,24.0,8.6,8.4,7.4,13.6
32779,2023-09-30 23:00:00,6010.0,39.500000,47.500000,Сб,NaN,NaN,NaN,NaN,NaN,...,17.6,30.099998,2920.399902,7.0,2.0,24.0,8.6,8.4,7.4,13.6


In [12]:
# t_set.to_excel("КС-15_set_v1.xlsx",index=None)